In [ ]:
suppressMessages(require(rms, quietly = TRUE, warn.conflicts = FALSE))
require(splines, quietly = TRUE)
require(plotly, quietly = TRUE, warn.conflicts = FALSE)
require(Hmisc, quietly = TRUE)
require(e1071, quietly = TRUE)
require(caret, quietly = TRUE)
require(BiodiversityR, quietly = TRUE)
require(logistf, quietly = TRUE)
require(rmarkdown, quietly = TRUE)

In [12]:
data <- read.csv("https://raw.githubusercontent.com/asiftanim/SOEN-6591---Software-Repository-Mining/main/project%20csv/all_merged.csv")

In [9]:
NA_THRESHOLD = 0.5
CORR_THRESHOLD = 0.7

myProjectList <- function(x, projects) {
  temp_list = vector("list", 0)
  
  for (i in 1:length(projects)) {
    temp_project_data <- x[x[,"Project"] == projects[i],]
    temp_list <- c(temp_list, list(temp_project_data))
  }
  return (temp_list)
}

myMissingData <- function(x) {  
  
  temp_naclus = naclus(x)
  plot(temp_naclus)
  
  if (sum(temp_naclus$sim < NA_THRESHOLD)) {
    temp_project_data_omitted <- na.omit(x)
    print(paste("MissingData: Small fraction found: Single imputation or discard. Discard is chosen. Discarded rows:",nrow(x)-nrow(temp_project_data_omitted), "/", nrow(x)))
  } else {
    # TODO: implement multiple imputation
    temp_project_data_omitted = x
    print(paste("MissingData: High fraction found: Multiple imputation needed! Discarded rows:",nrow(x)-nrow(temp_project_data_omitted), "/", nrow(x)))
  }
  return(as.data.frame(temp_project_data_omitted))
}

In [10]:
projects = as.vector(unique(data$Project))

data_list = myProjectList(data,projects)

In [ ]:
all_no_missing = vector("list", 0)
for (i in 1:length(projects)) {
  print(paste("Project:", projects[i]))
  temp_data = as.data.frame(data_list[i])
  
  temp_omitted = myMissingData(temp_data)
  all_no_missing <- c(all_no_missing, list(temp_omitted))
}

In [ ]:
all_list_omitted_1 = data_list
all_list_omitted = all_list_omitted_1

all_list_omitted = vector("list", 0)
for (i in 1:length(projects)) {
  print(paste("Project:", projects[i]))
  temp_data = as.data.frame(all_list_omitted_1[i])
  print(paste("nrow:",nrow(temp_data),"ncol:",ncol(temp_data)))

  #Don't do anything in here, repack only.

  all_list_omitted <- c(all_list_omitted, list(list(project=projects[i], data=temp_data)))
}

In [ ]:
source(file = "/Users/asiftanim/Documents/Concordia/4.\ Winter\ 2023/analysis_functions.R")
source(file = "/Users/asiftanim/Documents/Concordia/4.\ Winter\ 2023/construction_functions.R")

In [ ]:
modelSelectionAndNormalityAdjustment(all_list_omitted)

In [ ]:
dropToPredict = c("FilePath",  "Project","Name","Kind", "File", "POST.ReleaseDefects")

In [ ]:
data_list_low_corr = vector("list", 0)
  for (i in 1:length(projects)) {
    print(paste("Project:", projects[i]))
    #x = as.data.frame(data_list[i])
    
    independent = data[,!(names(data) %in% dropToPredict)]
    temp_project_varclus = varclus(~ ., data=independent, trans="abs")
    plot(temp_project_varclus)

    data_list_low_corr <- c(data_list_low_corr, list(temp_data))
  }



In [ ]:
drop2=c("POST.ReleaseDefects", "LogandThrowDetector")
independant=data[,!(names(data) %in% drop2)]

##########correlation 
correlations <- cor(independant, method="spearman") 
highCorr <- findCorrelation(correlations, cutoff = .75)

low_cor_names=names(independant[, -highCorr])

low_cor_data= independant[(names(independant) %in% low_cor_names)]
dataforredun=low_cor_data

#########start redun
redun_obj = redun (~. ,data = dataforredun ,nk =0)
after_redun= dataforredun[,!(names(dataforredun) %in% redun_obj $Out)]
redun_obj

In [ ]:
############ model
form=as.formula(paste("POST.ReleaseDefects>0~",paste(names(after_redun),collapse="+")))

model=glm(formula=form, data=log10(data+1), family = binomial(link = "logit"))
model
summary(model)